In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


In [2]:
modelo_red_neuronal = load_model("modelo_rostros.h5")

personas = ["Ichu", "Cami", "Saul", "Avl"]

In [ ]:
video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        print("Error al capturar video. Saliendo...")
        break

    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detección de rostros
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    rostros = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(100, 100))

    for x, y, w, h in rostros:
        rostro = gray[y:y+h, x:x+w]
        rostro = cv2.resize(rostro, (100, 100))
        rostro = rostro.reshape(1, 100, 100, 1) / 255.0  # Normalización

        # Predicción del modelo
        prediccion = modelo_red_neuronal.predict(rostro)
        etiqueta = np.argmax(prediccion)
        probabilidad = np.max(prediccion)
        nombre = personas[etiqueta]

        # Dibujar rectángulo y mostrar la etiqueta
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        texto = f"{nombre}: {probabilidad:.2f}"
        cv2.putText(frame, texto, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow("Reconocimiento de rostros", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()